In [ ]:
import os 
from datetime import date
from matplotlib import pyplot as plt
import numpy as np

# we need to set the current path to the directory
# containing the suite3d repository, this hack should
# do the trick
os.chdir(os.path.dirname(os.path.abspath("")))

from suite3d.job import Job
from suite3d import io
from suite3d import plot_utils as plot
from curation import run_preprocessing

In [ ]:
# update this to point to the demo data! 
tifs = io.get_tif_paths(r'C:\Users\suyash\UCL\s3d_demo_data\standard-2p')[:10]
for tif in tifs: print(tif)

In [ ]:
# Set the mandatory parameters
params = {
    # volume rate
    'fs': io.get_vol_rate(tifs[0]),
    
    # planes to analyze. 0 is typically the flyback, so we exclude it here
    'planes' : np.array([1,2,3,4]), 
    # number of planes recorded by scanimage, including the flyback
    'n_ch_tif' : 5,
    
    # Decay time of the Ca indicator in seconds. 1.3 for GCaMP6s. This example is for GCamP8m
    'tau' : 1.3,
    'lbm' : False, 
    'num_colors' : 2, # how many color channels were recorded by scanimage
    'functional_color_channel' : 0, # which color channel is the functional one
     # voxel size in z,y,x in microns
    'voxel_size_um' : (20, 1.5, 1.5),

    # number of files to use for the initial pass
    # usually, ~500 frames is a good rule of thumb
    # we will just use 200 here for speed
    'n_init_files' :  2,

    # 3D GPU registration - fast! 
    '3d_reg' : True,
    'gpu_reg' : True,
    
    # note : 3D CPU is not supported yet
    'subtract_crosstalk' : False, # turn off some lbm-only features
    'fuse_strips' : False, # turn off some lbm-only features

    
}

In [ ]:
# Create the job
job = Job(r'C:\Users\suyash\UCL\s3d_demo_data','demo-std', tifs = tifs,
          params=params, create=True, overwrite=True, verbosity = 3)

In [ ]:
job.run_init_pass()

In [ ]:
# If you have large tiffs, split the large tiffs into files of size 100 after registration
job.params['split_tif_size'] = 100

In [ ]:
# OPTIONAL: load and take a look at the reference image
summary = job.load_summary()
ref_img = summary['ref_img_3d']

# # view 1 plane at a time
plot.show_img(ref_img[3], figsize=(3,4))

# # interactive 3D viewer
plot.VolumeViewer(ref_img)


In [ ]:
job.register()

In [ ]:
corr_map = job.calculate_corr_map()

In [ ]:
res = job.load_corr_map_results()
vmap = res['vmap']

In [ ]:
job.params['patch_size_xy'] = (550, 550)
# for speed, only segment a single patch
job.segment_rois()

In [ ]:
job.compute_npil_masks()
traces = job.extract_and_deconvolve()

In [ ]:
job.export_results(r"C:\Users\suyash\UCL\s3d_demo_data\output",result_dir_name='rois')

To take a look at the outputs in napari, navigate to the suite3d directory in a command shell and run the following:
```
python curation.py curation --output_dir /path/to/output/rois
```

In [ ]:
# Run the curation pipeline
run_preprocessing.preprocess(job.job_dir)